In [1]:


def pauli_tabular(a, b):
    if a == 'i':
        return 1, b
    if b == 'i':
        return 1, a
    if a == b:
        return 1, 'i'
    if a == 'x':
        if b == 'y':
            return 1j, 'z'
        if b == 'z':
            return -1j, 'y'
    if a == 'y':
        if b == 'x':
            return -1j, 'z'
        if b == 'z':
            return 1j, 'x'
    if a == 'z':
        if b == 'x':
            return 1j, 'y'
        if b == 'y':
            return -1j, 'x'
    return 
        
def stabilizer_tabular(pauli, gate, param = 0):
    if pauli == 'i':
        return 1, 'i'
    if gate == 'h':
        if pauli == 'x':
            return 1, 'z'
        if pauli == 'y':
            return -1, 'y'
        if pauli == 'z':
            return 1, 'x'
    elif gate == 's':
        if pauli == 'x':
            return 1, 'y'
        elif pauli == 'y':
            return -1, 'x'
        elif pauli == 'z':
            return 1, 'z'
    elif gate == 't':
        if pauli == 'x':
            return [1/np.sqrt(2), 1/np.sqrt(2)], ['x','y']  # X -> (X + Y) / √2
        elif pauli == 'y':
            return [1/np.sqrt(2), -1/np.sqrt(2)], ['y','x']  # Y -> (Y - X) / √2
        elif pauli == 'z':
            return 1, 'z'  # Z -> Z
    elif gate == 'rx':  #
        if pauli == 'x':
            return 'x'
        elif pauli == 'y':
            return [np.cos(param), np.sin(param)], ['y','z']
        elif pauli == 'z':
            return [np.cos(param), -np.sin(param)], ['z','y']
    elif gate == 'ry':
        if pauli == 'x':
            return [np.cos(param), np.sin(param)], ['z','x']
        elif pauli == 'y':
            return 1, 'y'
        elif pauli == 'z':
            return [np.cos(param), -np.sin(param)], ['x','z']
    elif gate == 'rz': 
        if pauli == 'x':
            return [np.cos(param), np.sin(param)], ['x','y']
        elif pauli == 'y':
            return [np.cos(param), -np.sin(param)], ['y','x']
        elif pauli == 'z':
            return 1, 'z'
        
    elif gate == 'cx':  # Controlled-X gate
        if pauli == 'ix':
            return 1, 'ix'
        elif pauli == 'xi':
            return 1, 'xx'
        elif pauli == 'iy':
            return 1, 'zy'
        elif pauli == 'yi':
            return 1, 'yx'
        elif pauli == 'iz':
            return 1, 'zz'
        elif pauli == 'zi':
            return 1, 'zi'
        elif pauli == 'xx':
            return 1, 'xi'
        elif pauli == 'xy':
            return 1, 'yz'
        elif pauli == 'xz':
            return -1, 'yy'
        elif pauli == 'yx':
            return 1, 'yi'
        elif pauli == 'yy':
            return -1, 'xz'
        elif pauli == 'yz':
            return 1, 'xy'
        elif pauli == 'zx':
            return 1, 'zx'
        elif pauli == 'zy':
            return 1, 'iy'
        elif pauli == 'zz':
            return 1, 'iz'
        elif pauli == 'ii':
            return 1, 'ii'
    else:
        return None

In [2]:
class PauliWord:
    # Example: 2*ixz
    def __init__(self, scalar, word: str):
        self.scalar = scalar
        self.word = word
    def duplicate(self):
        p = PauliWord(self.scalar, self.word)
        return p
    def create_empty(self):
        p = PauliWord(self.scalar, self.word)
        p.word = 'i'*len(self.word)
        return p
    def get(self,index):
        return self.word[index]
    def update(self, index, character):
        self.word = self.word[:index] + character + self.word[index+1:]
    def update_scalar(self, scalar):
        self.scalar = self.scalar * scalar
    def update_word(self, word):
        self.word = word
    def return_matrix(self):
        matrix = self.scalar * kr(self.word)
        return matrix
    def multiply(self, other):
        # Multiply two PauliWords
        word_temp = self.create_empty()
        word_temp.scalar = self.scalar * other.scalar
        for i, _ in enumerate(self.word):
            scalar, character = pauli_tabular(self.word[i], other.word[i])
            word_temp.scalar = word_temp.scalar * scalar
            word_temp.word =  word_temp.word[:i] + character +  word_temp.word[i+1:]
        return word_temp
    def __str__(self):
        return str(self.scalar) + '*' + self.word

class PauliTerm:
    def __init__(self, words):
        self.words: list[PauliWord] = words
    def __str__(self):
        return ' + '.join([str(word) for word in self.words])
    def return_matrix(self):
        matrix = np.zeros((2**len(self.words[0].word), 2**len(self.words[0].word)), dtype=complex)
        for word in self.words:
            matrix += word.return_matrix()
        return matrix
    def reduce(self):
        # Example: P = [1*zxx, 1*yzi, 1j*zxx] -- reduce() --> [(1+1j)*zxx, 1*yzi]
        # Example: P = [1*zxx, 1*yzi, -1*zxx] -- reduce() --> [1*yzi]
        element_sum = {}
        for pauli_word in self.words:
            scalar, string = pauli_word.scalar, pauli_word.word
            if string in element_sum:
                element_sum[string] += scalar
            else:
                element_sum[string] = scalar

        self.words = [PauliWord(value, key) for key, value in element_sum.items() if value != 0]
        return
    def multiply(self, other):
        # Multiply two PauliTerms
        new_terms = []
        for i in self.words:
            for j in other.words:
                new_terms.append(i.multiply(j))
        return PauliTerm(new_terms)
    def map(self, gate, index, param):
        # Example: xii -- (h, 0) --> [z]ii
        # Example: xxx -- (t, 0) --> [x']xx = [1/sqrt(2) (x + y)]xx
        # Example: zxz -- (cx, [0,2]) --> [i]x[z]
        # This function will map a Pauli term to another Pauli term
        # Example: xii + xxx -- (h, 0) --> [z]ii + [z]xx
        # Two keys: only act on coressponding qubits, and independence between pauli strings
        num_words = len(self.words)
        for j in range(num_words):
            # Process on a single Pauli string
            
            if gate == 'cx':
                # Index will be [control, target]
                # Word will be [word_control, word_target]
                out_scalar, output_word = stabilizer_tabular(
                    self.words[j].get(index[0]) + self.words[j].get(index[1]), gate)
                self.words[j].update(index[0], output_word[0])
                self.words[j].update(index[1], output_word[1])
                self.words[j].update_scalar(out_scalar)
            else:
                # Index will be just a scalar
                if gate in ['rx', 'ry', 'rz']:
                    out_scalar, output_word = stabilizer_tabular(
                        self.words[j].get(index), gate, param)
                else:
                    out_scalar, output_word = stabilizer_tabular(
                        self.words[j].get(index), gate)
                if type(output_word) == list:
                    self.words.append(self.words[j].duplicate())
                    self.words[j].update(index, output_word[0])
                    self.words[j].update_scalar(out_scalar[0])
                    self.words[-1].update(index, output_word[1])
                    self.words[-1].update_scalar(out_scalar[1])
                else:
                    self.words[j].update(index, output_word)
                    self.words[j].update_scalar(out_scalar) 
        self.reduce()
        return
class StabilizerGenerator:
    def __init__(self, num_qubits):
        # Example: if the system is 3 qubits, then the stabilizer group will be {Z0, Z1, Z2}
        # or {zii, izi, iiz}
        self.num_qubits = num_qubits
        init_words = ['i' * i + 'z' + 'i' * (num_qubits - i - 1) for i in range(num_qubits)]
        init_stabilizers = [PauliTerm([PauliWord(1, word)]) for word in init_words]
        self.stabilizers: list[PauliTerm] = init_stabilizers
        self.ps: list[PauliTerm] = []
    def __str__(self):
        string = '<'
        for i in self.stabilizers[:-1]:
            string += str(i) + '\n'
        return string + str(self.stabilizers[-1]) + '>'
    def reduce(self):
        for i, _ in enumerate(self.stabilizers):
            self.stabilizers[i].reduce()
        return
    def map(self, gate: str, index, param = 0):
        for i, _ in enumerate(self.stabilizers):
            self.stabilizers[i].map(gate, index, param)
        return
    def generate_p(self):
        def get_subsets(n):
            from itertools import chain, combinations
            input_set = set(range(n))
            subsets = list(chain.from_iterable(combinations(input_set, r) for r in range(len(input_set) + 1)))
            result = [set(subset) if subset else set() for subset in subsets] 
            return result
        sets = get_subsets(len(self.stabilizers))
        self.ps = []
        for indices in sets:
            p = PauliTerm([PauliWord(1, 'i'*self.num_qubits)])
            for j in indices:
                p = p.multiply(self.stabilizers[j])
            self.ps.append(p)
        return
    def generate_density_matrix(self):
        if len(self.ps) == 0:
            self.generate_p()
        density_matrix = np.zeros((2**self.num_qubits, 2**self.num_qubits), dtype=complex)
        for p in self.ps:
            density_matrix += p.return_matrix()
        return (1/2**self.num_qubits)*density_matrix
StabilizerGenerator.h = lambda self, index: self.map('h', index)
StabilizerGenerator.s = lambda self, index: self.map('s', index)
StabilizerGenerator.t = lambda self, index: self.map('t', index)
StabilizerGenerator.cx = lambda self, index: self.map('cx', index)
StabilizerGenerator.rx = lambda self, param, index: self.map('rx', index, param)
StabilizerGenerator.ry = lambda self, param, index: self.map('ry', index, param)
StabilizerGenerator.rz = lambda self, param, index: self.map('rz', index, param)


In [3]:
p1 = PauliWord(1j, 'zxx')
p2 = PauliWord(1, 'zxx')
p3 = PauliWord(1, 'yzi')
t1 = PauliTerm([p1, p3])
stb = StabilizerGenerator(2)
stb.h(0)
stb.cx([0,1])
stb.t(0)
stb.cx([0,1])
stb.h(0)
stb.rx(np.pi/3, 0)


In [4]:
stb.generate_p()
for i in stb.ps:
    print(i)



1*ii
-0.2588190451025207*zi + -0.9659258262890682*yi
1*iz
-0.2588190451025207*zz + -0.9659258262890682*yz


In [5]:
dm = stb.generate_density_matrix()
print(np.round(dm,2))

[[ 0.37+0.j    0.  +0.j    0.  +0.48j  0.  +0.j  ]
 [ 0.  +0.j   -0.  +0.j    0.  +0.j    0.  +0.j  ]
 [ 0.  -0.48j  0.  +0.j    0.63+0.j    0.  +0.j  ]
 [ 0.  +0.j    0.  +0.j    0.  +0.j    0.  +0.j  ]]
